## This notebook is ran once to generate the qza files used for expectedresults in unit tests for establish_dataset

In [3]:
from qiime2 import Artifact
from qiime2 import Metadata
from qiime2.plugins.demux.methods import emp_single
from qiime2.plugins.quality_filter.methods import q_score
from qiime2.plugins.deblur.methods import denoise_16S
import pandas as pd
import wget
import biom
import os

print("Importing data for expected")
exp_seqs = Artifact.import_data("EMPSingleEndSequences",
                                     "data/mock-3/emp-single-end-sequences")

print("Importing metadata for expected")
exp_barcode_metadata = \
    Metadata.load("data/mock-3/sample-metadata.tsv")

print("Demuxing for expected")
exp_demux, = emp_single(exp_seqs,
                             exp_barcode_metadata.get_category(
                                 "BarcodeSequence"))

Importing data for expected
Importing metadata for expected
Demuxing for expected


In [5]:
print("Quality filtering (with default params)")
demuxed_qfiltered, demuxed_qf_stats = q_score(exp_demux)

print("Deblur-ing with trim length {:d}".format(150))
deblurred, repseq, deblur_stats = denoise_16S(demuxed_qfiltered, 150,
                                              hashed_feature_ids = False)

Quality filtering (with default params)
Deblur-ing with trim length 150


In [7]:
# generate post trimmed expected
print("Trimming post-demuxed seqs to {:d}".format(100))

demuxed_biom = deblurred.view(biom.Table)
post_trimmed_biom = demuxed_biom.collapse(lambda i, m:i[:100], axis="observation")
post_trimmed = Artifact.import_data("FeatureTable[Frequency]", post_trimmed_biom);

Trimming post-demuxed seqs to 100


In [15]:
deblurred.save("deblurred_150nt.qza")

'deblurred_150nt.qza'

In [16]:
deblurred_cpy = Artifact.load("deblurred_150nt.qza")

In [17]:
deblurred_cpy == deblurred

True

In [18]:
post_trimmed.save("deblurred_100nt_post_trim.qza")

'deblurred_100nt_post_trim.qza'

In [21]:
pt_cpy = Artifact.load("deblurred_100nt_post_trim.qza")
pt_cpy

<artifact: FeatureTable[Frequency] uuid: cf5de485-0ef3-491e-a8e6-1708f7feedcd>

In [4]:
exp_demux.save("data/mock-3/exp_demux.qza")

'data/mock-3/exp_demux.qza'

In [ ]:
exp_barcode_metadata.s